# CS 181 - Final Project: Web Service API's 
## Token Acquisition Notebook

Authors: Lucas Tran & Amna Khalid

Date: 12/16/2019

### Note: Do not run this notebook since you do not have the first authorization

In [1]:
import json
import requests

In [2]:
#load in the json file that has users' data
with open("spotify.json", "r") as file:
    creds = json.load(file)

In [3]:
def authoURL():
    '''
    Paramters: none
    Purpose: this function creates a authorization url. Using this url, people can sign in 
             and grant access to their spotify data
    Return: p.url - the authorization url
    '''
    #create an authorization url
    protocol = "https"
    location = "accounts.spotify.com"
    auth_resource = "/authorize"
    authurl_fmt = "{}://{}{}"
    authurl = authurl_fmt.format(protocol, location, auth_resource)

    #create a url query using information of main user from the json file
    urlquery = {}
    urlquery['client_id'] = creds['spotify']['appid']
    urlquery['redirect_uri'] = creds['spotify']['redirect_uri']
    urlquery['response_type'] = 'code'
    urlquery['state'] = '1234567890'
    urlquery['scope'] = "user-follow-read user-top-read user-read-recently-played user-library-read"
    urlquery
    
    #sending get request to get one token
    session = requests.Session()
    p = requests.Request('GET', authurl, params=urlquery).prepare()
    
    return p.url

In [4]:
def getInitialTokens(code):
    '''
    Paramters: code - the access code from the url
    Purpose: perform a post request using the code from above url to get the access token and the refresh token
             scope decides what data we get from this post request
    Return: resp.json() - json file that contains access token and refresh token
    '''
    #perform a post request using the code from above url to get the access token and the refresh token
    #scope decides what data we get from this post request
    url = 'https://accounts.spotify.com/api/token'
    bodyparams = {'client_id':'9e6455bc1d7d454b9e16ba1001252001',
                  'client_secret':'f6eb1de058334c249f98b2df14a09833',
                  'grant_type': 'authorization_code', 
                  'code': code,
                  'redirect_uri':'https://localhost/callback/',
                  'scope': 'user-follow-read user-top-read user-read-recently-played user-library-read'}
    bodyparams['code'] = code
    resp = requests.post(url, data=bodyparams)
    return resp.json()

In [5]:
def getNewAccess():
    
    '''
    Paramters: none
    Purpose: this functions get the updated version of access token 
    using the refresh token given by the above function
    Return: 
        - username: username list
        - access: access token list
    '''
    
    refresh = []
    username = []
    url1 = 'https://accounts.spotify.com/api/token'
    access = [] #create a blank list to store the new access token 
    for item in creds['spotify']['users']:
        refresh.append(creds['spotify']['users'][item]['refresh_token'])
        username.append(item)

    #create a generalize body parameters with refresh token left blank so we can use it 
    #over and over again
    bodyparams = {'client_id':'9e6455bc1d7d454b9e16ba1001252001',
                  'client_secret':'f6eb1de058334c249f98b2df14a09833',
                  'grant_type': 'refresh_token', 
                  'refresh_token': '',
                  'redirect_uri':'https://localhost/callback/',
                  'scope': 'user-follow-read user-top-read user-read-recently-played user-library-read'}
    #using the refresh token from the json file, get the access token and append that acces token to access list
    for item in refresh:
        bodyparams['refresh_token'] = item
        resp = requests.post(url1, data=bodyparams)
        access.append(resp.json()['access_token'])
    return username, access

In [6]:
def main():
    '''
    Parameters: none
    Purpose: call all the functions above 
    Return: none
    '''
    url = authoURL()
    print("Authorization url:",url)
    
    #the code is outdated so this will return invalid grant if you run it
    #gets first token
    code = 'AQCDMNqaQ4wwOjFdN-oQKib7hjKupi86v4NEgUKMlqy5nnh5VwZUOnPQoswAwcJSFcx_RZCUVZDCNKoP_kvi1RGRrYgIhiQFNOnFXwMLaPf-cEB3C_Esx7Mm6X0_LvRw3tM4CaqMWDF-hrSaNfWtWZDAArQAoFenmKI-I1yRVnt_q3oxGfycfUTB-xYaKK3GS8Y9w8eeqfZo0zf_a0z4IwlornenMAmxh7vD-OAeQVrD9wEOMSCnC23dLzKw5ncrGoMb5G8PPwklcZ6wTZ9WtSIh-no8KmAcy3vBgixrSpw'
    resp = getInitialTokens(code)
    print ("Initial access token and refresh token:",resp)
    
    username, access = getNewAccess()
    print("List of username:",username)
    print("List of access token:",access)

In [7]:
main()

Authorization url: https://accounts.spotify.com/authorize?client_id=9e6455bc1d7d454b9e16ba1001252001&redirect_uri=https%3A%2F%2Flocalhost%2Fcallback%2F&response_type=code&state=1234567890&scope=user-follow-read+user-top-read+user-read-recently-played+user-library-read
Initial access token and refresh token: {'access_token': 'BQCixJZMBtEwj1hEjKpk7OvfotiK0pKfPhs49HTlozqBy1Zfnpw30FRc171BsLG0NNuLmX6FhjIhO47WbuEau-PWryDAXtVkWUxDDdttG6Y-k3EwHLDC3UcuzQkqJv_O_NwJypHCKnwnSXelcqsyhYmUM0SrllRuXKUF3FekZAFtDcBLtrNPX_P4wiYQmg', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQDoXw5x9MTvcIF3CzHjcFRkQrkk7AfrAKAaK206ySHnRtCa7f4vh462INxpEHL6_ivxUJY9B6a2pmZoliiazNg_tnQ6wl7EnE3ndaeWBdJw5lKmo629DAvLxlLMchZeJ1c', 'scope': 'user-library-read user-follow-read user-read-recently-played user-top-read'}
List of username: ['kelly', 'giorgi', 'liam', 'issac', 'brandon', 'jessie', 'tess', 'justin', 'olivia', 'devan', 'diamond', 'Ziang', 'Jessica', 'Amna', 'Katie']
List of access token: ['BQBKa5H68KqE